# Identify subset of examples for direct quotes

## Load & filter annotation data

**Goal:** Find a subset of pages with direct quotes to be use for testing and experimentation.

In [ ]:
import marimo as mo
import polars as pl

In [ ]:
# load all annotation data files
df = pl.read_csv("data/annotation/*.csv")
# limit to the columns we care about
df = df.select(
    pl.col("UUID", "FILE", "QUOTE_TRANSCRIPTION", "ANCHOR", "TAGS", "COMMENTS")
)
# turn char-offset into numeric start index, calculate end index
df = (
    df.with_columns(start_index=pl.col("ANCHOR").str.slice(12).cast(dtype=int))
    .with_columns(
        end_index=pl.col("start_index").add(
            pl.col("QUOTE_TRANSCRIPTION").str.len_chars()
        )
    )
    .drop("ANCHOR")
)
df

<marimo-table data-initial-value='[]' data-label='null' data-data='"./@file/18246-37715080-VZmlQl47.json"' data-total-rows='347' data-total-columns='7' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["UUID", ["string", "str"]], ["FILE", ["string", "str"]], ["QUOTE_TRANSCRIPTION", ["string", "str"]], ["TAGS", ["string", "str"]], ["COMMENTS", ["string", "str"]], ["start_index", ["integer", "i64"]], ["end_index", ["integer", "i64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

Filter to the subset of annotations that have direct references to the texts that are in scope for this phase.

In [ ]:
# filter to the subset with direct quotes from texts of interest
# - we want tag only, no qualification such as |Title Reference or |Concept Reference
quotes_df = df.filter(
    pl.col("TAGS").is_in(["Kapital", "Manifest der Kommunistischen Partei"])
)
quotes_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"./@file/9210-37715080-E4usOkWn.json"' data-total-rows='45' data-total-columns='7' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["UUID", ["string", "str"]], ["FILE", ["string", "str"]], ["QUOTE_TRANSCRIPTION", ["string", "str"]], ["TAGS", ["string", "str"]], ["COMMENTS", ["string", "str"]], ["start_index", ["integer", "i64"]], ["end_index", ["integer", "i64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
quotes_df.group_by("FILE").agg(
    pl.col("TAGS").count().alias("count"),
    pl.col("TAGS").unique().str.join("|"),
)

<marimo-table data-initial-value='[]' data-label='null' data-data='"./@file/498-37715080-Q1kmF4vB.json"' data-total-rows='6' data-total-columns='3' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["FILE", ["string", "str"]], ["count", ["integer", "u32"]], ["TAGS", ["string", "str"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

`1896-97aCLEAN.txt` has the most direct quotes and has some from both Marx texts; let's use that one.

In [ ]:
# 1896-97aCLEAN.txt has the most direct quotes and has some from both texts
quote_subset_df = quotes_df.filter(pl.col("FILE").eq("1896-97aCLEAN.txt"))
quote_subset_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"./@file/9210-37715080-xhYgmDCJ.json"' data-total-rows='16' data-total-columns='7' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["UUID", ["string", "str"]], ["FILE", ["string", "str"]], ["QUOTE_TRANSCRIPTION", ["string", "str"]], ["TAGS", ["string", "str"]], ["COMMENTS", ["string", "str"]], ["start_index", ["integer", "i64"]], ["end_index", ["integer", "i64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

## Load article text and chunk roughly into pages

Load the text contents of the file for the annotations we want to use and make sure we can match pages of text to the annotated text.

In [ ]:
# load text file and chunk into pages

with open("data/text/1896-97aCLEAN.txt") as inputfile:
    text = inputfile.read()
    pages = text.split("\n\n\n")

page_df = pl.DataFrame(data={"text": pages}).with_row_index()
# add field to calculate text length
page_df = page_df.with_columns(text_length=pl.col("text").str.len_chars())
page_df

<marimo-table data-initial-value='[]' data-label='null' data-data='"./@file/70366-37715080-ZXXf7SWw.json"' data-total-rows='885' data-total-columns='3' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["index", ["integer", "u32"]], ["text", ["string", "str"]], ["text_length", ["integer", "u32"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
# calculate start index for each page based on preceding text length + split characters
page_start_indices = []
# first page index is zero
current_index = 0
for page in page_df.iter_rows(named=True):
    # add current page index to the list
    page_start_indices.append(current_index)
    # add length of this place plus characters used to split
    current_index += page["text_length"] + 3

In [ ]:
# add page start index to the dataframe of page text and calculate end of page
page_df_start = page_df.insert_column(
    3, pl.Series("page_start", page_start_indices)
).with_columns(page_end=pl.col("page_start").add(pl.col("text_length")))
page_df_start

<marimo-table data-initial-value='[]' data-label='null' data-data='"./@file/70716-37715080-FenQqbFl.json"' data-total-rows='885' data-total-columns='5' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["index", ["integer", "u32"]], ["text", ["string", "str"]], ["text_length", ["integer", "u32"]], ["page_start", ["integer", "i64"]], ["page_end", ["integer", "i64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

Spot check alignment of page text and annotation text.

In [ ]:
# check text index matching
first_row = quotes_df.row(0, named=True)
first_row

In [ ]:
text_substring = text[first_row["start_index"] : first_row["end_index"]]
print(text_substring)

„die Arbeiter¬
klasse nicht die fertige Staatsmaschine einfach in Besitz nehmen und für ihre
eigenen Zwecke in Bewegung setzen kann“


In [ ]:
print(first_row["QUOTE_TRANSCRIPTION"])

„die Arbeiter¬ klasse nicht die fertige Staatsmaschine einfach in Besitz nehmen und für ihre eigenen Zwecke in Bewegung setzen kann“


In [ ]:
text_substring.replace("\n", " ") == first_row["QUOTE_TRANSCRIPTION"]

True

The text in the exported annotation does not include newlines, but once we're calculating text length and start/end index
correctly, we do have matching text (other than newlines).

## Combine page text with selected annotations

In [ ]:
# join subset of quotes with page text; rename page text columns for clarity
quote_subset_pages = quote_subset_df.join_where(
    page_df_start.rename({"text": "page_text", "index": "page_index"}),
    pl.col("start_index") >= pl.col("page_start"),
    pl.col("start_index") <= pl.col("page_end"),
)
quote_subset_pages

<marimo-table data-initial-value='[]' data-label='null' data-data='"./@file/47946-37715080-gkGth3jt.json"' data-total-rows='16' data-total-columns='12' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["UUID", ["string", "str"]], ["FILE", ["string", "str"]], ["QUOTE_TRANSCRIPTION", ["string", "str"]], ["TAGS", ["string", "str"]], ["COMMENTS", ["string", "str"]], ["start_index", ["integer", "i64"]], ["end_index", ["integer", "i64"]], ["page_index", ["integer", "u32"]], ["page_text", ["string", "str"]], ["text_length", ["integer", "u32"]], ["page_start", ["integer", "i64"]], ["page_end", ["integer", "i64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

Check alignment of text between quotes and page text.

In [ ]:
# iterate over quotes and output to check that we're getting the correct content

for quote in quote_subset_pages.iter_rows(named=True):
    print(quote["QUOTE_TRANSCRIPTION"])
    page_start_index, page_end_index = (
        quote["start_index"] - quote["page_start"],
        quote["end_index"] - quote["page_start"],
    )
    print(f"{quote['TAGS']} (article {quote['start_index']}:{quote['end_index']} / page {page_start_index}:{page_end_index}) ")
    if page_end_index > len(quote['page_text']):
        print("*** quote end index is larger than page content")
    print(quote["page_text"][page_start_index:page_end_index])
    print("\n")

Zur Lösung dieses Widerspruchs" fährt er fort, „bedarf es noch vieler Mittelglieder.“ Er versprach, diese Lösung später zu geben.
Kapital (article 2262872:2263001 / page 2802:2931) 
Zur Lösung
dieses Widerspruchs" fährt er fort, „bedarf es noch vieler Mittelglieder.“ Er
versprach, diese Lösung später zu geben.


daß dies Gesetz offenbar, aller auf den Augenschein gegründeten Erfahrung widerspricht". „
Kapital (article 2262782:2262872 / page 2712:2802) 
daß dies Gesetz offenbar,
aller auf den Augenschein gegründeten Erfahrung widerspricht". „


1 Vergl. „Kapital“, Bd. I, S. 329, Aum. 75, und Roscher, „System der Volkswirth¬ schaft, 1. Auflage (1854½), S. 359, 360, 361.
Kapital (article 1843563:1843689 / page 3696:3822) 
1 Vergl. „Kapital“, Bd. I, S. 329, Aum. 75, und Roscher, „System der Volkswirth¬
schaft, 1. Auflage (1854½), S. 359, 360, 361.


Marx hat ferner die Ausführung von Roscher total auf den Kopf gestellt¬ und endlich den Anschein erweckt, als ob sein Werthbegriff etwas wefen

Some pages have more than one quote, but in this set the quotes on the same page are from the same Marx text.

In [ ]:
# which pages have more than one quote?
quote_subset_pages.group_by("page_index").agg(
    pl.col("TAGS").count().alias("count"),
    pl.col("TAGS").unique().str.join("|"),
).filter(pl.col("count").gt(1))

<marimo-table data-initial-value='[]' data-label='null' data-data='"./@file/332-37715080-SOmmQ8bK.json"' data-total-rows='6' data-total-columns='3' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["page_index", ["integer", "u32"]], ["count", ["integer", "u32"]], ["TAGS", ["string", "str"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='false' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

See the quotes in the context of the page. Use the slider to move between different quotes.

In [ ]:
quote_slider = mo.ui.slider(
    start=0,
    stop=quote_subset_pages.height - 1,
    step=1,
    label="Quote from subset",
)
quote_slider

In [ ]:
def show_page(quote):
    page_start_index, page_end_index = (
        quote["start_index"] - quote["page_start"],
        quote["end_index"] - quote["page_start"],
    )
    # at least one page includes an asterisk; escape so we don't get unintentional italics
    before_quote = quote["page_text"][0:page_start_index].replace("*", r"\*")
    quote_text = quote["page_text"][page_start_index:page_end_index].replace(
        "*", r"\*"
    )
    after_quote = quote["page_text"][page_end_index:].replace("*", r"\*")

    return mo.md(f"""
    Tag: {quote["TAGS"]}<br/>
    Page index: {quote["page_index"]} (article: {page_start_index}:{page_end_index} page: {page_start_index}:{page_end_index})

    {before_quote}**{quote_text}**{after_quote}
    """)


show_page(quote_subset_pages.row(quote_slider.value, named=True))

Tag: Kapital 
 Page index: 661 (article: 2802:2931 page: 2802:2931) 
 Die Neue Zeit.
612
—
Männer" habet in solchen gallen so gut wie immer für die Reaktion und gegen¬
die Sozialdemokratie entschieden, unter schmunzelndem Schweigen des Herrn Richter
Einerseits weiß er, daß die Junker nach dem Grundsatz leben: Hand wird nur
von Hand gewaschen, daß sie bei den Stichwahlen den Freisinn aufsttzen lassen,
rseits weiß er, daß die Sozialdemokraten prin¬
wenn er sie aufsitzen läßt, ander.
zipielle Politik treiben und in den Stichwahlen zwischen Freisinn und Reaktion
immer dem „kleineren Uebel“ den Vorzug geben. Herr Richter will, daß die
freisinnige Fraktion des Reichstags fortbesteht wie bisher: halb als Hilfstruppen,
der Reaktion, halb als Kanonenfutter der Sozialdemokratie. Und würde man
ihm sagen, das sei doch ein sehr schäbiges Dasein, so würde der große Denker
achselzuckend antworten: Lieber sehr schäbig, als gar nicht leben!"
Der schöne Gedanke der „liberalen Phalanx" ließe sich nur etwa dann
denken, wenn das berühmte „deutsche Bürgerthum" sich aus eigener Kraft auf¬
raffte, sowohl der Klique Richter=Träger als auch der Klique Rickert=Barth den
Laufpaß gäbe, sich den Kopf frei machte von dem „allgemeinen Phrafennebel,
der kapitalistischen Presse und endlich einmal, was ihm Lassalle schon vor dreißig,
Jahren gerathen hat, selbst denken und selbst handeln kernte. Aber natürlich
ist dieser Gedanke die reine Phantasmagorie, genau dieselbe Phantasmagorie,
welche das berühmte „deutsche Bürgerthum“ selbst ist. Die liberale Bourgeoisie,
ist ein Offizierkorps ohne Heer, und findet keine Massen mehr, die ihre Schlachten¬
schlagen. Das Kleinbürgerthum ist in seinen rückständigeren Elementen zum Anti¬
semittsmus, in seinen vorgeschritteneren Elementen zur Sozialdemokratie ab¬
geschwenkt. Das industrielle Proletariat läßt sich nirgends mehr durch kapitalistischer
Redensartet nasführen, und das ländliche Proletariat lebt noch im geistigen
Banne der Junker und Pfaffen, aus dem es einmal die Sozialdemokratie lösen¬
wird, aber die liberale Bourgeoisie niemals lösen kann.
Somit wird die „liberale Phalanx“ im Reiche der „Wahldichtung" und
der „Trittksprüche" bleiben, und der freisinnige Wahlkampf wird in den Bahnen
verlaufen, die ihm Herr Richter vorgezeichnet hat. Wobei für die übrige Menschheit,
wenigstens der Trost abfällt, daß auch der verbohrteste Manchestermann durch
fortgesetzte Schläge etwas lernen kann, wenn auch nur etwas sehr Schäbiges. 
 NEW_DOCUMENT 
Lafargue,_1897_15:01_531_Die-ökonomischen-Funktionen-der-Börse. 
 ///// Die ökonomischen Funktionen der Börse. 
Ein Beitrag zur Werththeorie [1] ////// Paul Lafargue 
 Nachdem Marx im ersten Bande des „Kapital“, Kapitel X, das Gesetz,
des Mehrwerths festgestellt hatte, fügte er sogleich hinzu, „daß dies Gesetz offenbar,
aller auf den Augenschein gegründeten Erfahrung widerspricht". „ Zur Lösung
dieses Widerspruchs" fährt er fort, „bedarf es noch vieler Mittelglieder.“ Er
versprach, diese Lösung später zu geben. 
Die wenigen Nationalökonomen, denen diese Stelle auffiel, setzten ihre
Zuversicht auf diesen ihnen unlösbar scheinenden Widerspruch, der nach ihrer
Ansicht die Theorie zu Falle bringen mußte. Mehrere hofften, daß der Theore¬
tiker des Werthes dort mit seiner Dialektik und seinem Kommumnismus scheitern.
werde, denen, wie sie überzeugt waren, jede wissenschaftliche Grundlage fehle.
Herr Loria, der geniale Entdecker so mancher schon von Marx entdeckten
Theorien, ging so weit, zu behaupten, daß derselbe, um seine Ohumacht nicht
einzugestehen, sich entschlossen hätte, die beiden Bände, welche sein ökonomisches

Save the identified subset of quotes and associated page text for use in other experiments.

In [ ]:
quote_subset_pages

<marimo-table data-initial-value='[]' data-label='null' data-data='"./@file/47946-37715080-dTQ7wVQZ.json"' data-total-rows='16' data-total-columns='12' data-banner-text='""' data-pagination='true' data-page-size='10' data-field-types='[["UUID", ["string", "str"]], ["FILE", ["string", "str"]], ["QUOTE_TRANSCRIPTION", ["string", "str"]], ["TAGS", ["string", "str"]], ["COMMENTS", ["string", "str"]], ["start_index", ["integer", "i64"]], ["end_index", ["integer", "i64"]], ["page_index", ["integer", "u32"]], ["page_text", ["string", "str"]], ["text_length", ["integer", "u32"]], ["page_start", ["integer", "i64"]], ["page_end", ["integer", "i64"]]]' data-show-filters='true' data-show-download='true' data-show-column-summaries='true' data-row-headers='[]' data-has-stable-row-id='false' data-lazy='false' data-preload='false'>

In [ ]:
quote_subset_pages.write_csv("data/subset/direct_quotes.csv", include_bom=True)